# Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from Kaggle, and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from Kaggle
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.

   - Remember that for multi-class classification you need a softamx activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Trian your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaning the activation functions on the hidden layers, etc.
8. Load in Kaggle's `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# read in data
training_data = pd.read_csv('./train.csv')

In [3]:
training_data.head()
#targets
sorted(training_data.label.unique())


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
#target and features
y = training_data['label']
feats= [ col for col in training_data.columns if col != 'label' ]
X = training_data[feats]

In [5]:
X.shape
y.shape


(42000,)

In [6]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=55)

In [7]:
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(10500,)

In [8]:
#normalizing hexadecimal
X_train /= 255


In [9]:
X_test /= 255

In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

/Users/scottsanowitz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [11]:
#target matrix for nueral net
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [12]:
y_train.shape
y_test.shape

(10500, 10)

In [13]:
#model
model = Sequential()

In [14]:
# layers,
n_input = X_train.shape[1]
n_hidden = n_input
#hidden layer
model.add(Dense(n_input, activation='relu', input_dim=n_input))
# out put layer with softmax activation
model.add(Dense(10, activation='softmax'))

In [15]:
# compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [16]:
#fit
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs =50, batch_size=2000)

Train on 31500 samples, validate on 10500 samples
Epoch 1/50
31500/31500 [==============================] - 3s 87us/step - loss: 1.0081 - acc: 0.7369 - val_loss: 0.4217 - val_acc: 0.8803
Epoch 2/50
31500/31500 [==============================] - 3s 81us/step - loss: 0.3520 - acc: 0.8965 - val_loss: 0.3058 - val_acc: 0.9156
Epoch 3/50
31500/31500 [==============================] - 3s 82us/step - loss: 0.2721 - acc: 0.9210 - val_loss: 0.2596 - val_acc: 0.9288
Epoch 4/50
31500/31500 [==============================] - 3s 81us/step - loss: 0.2310 - acc: 0.9347 - val_loss: 0.2304 - val_acc: 0.9360
Epoch 5/50
31500/31500 [==============================] - 3s 82us/step - loss: 0.2010 - acc: 0.9439 - val_loss: 0.2059 - val_acc: 0.9424
Epoch 6/50
31500/31500 [==============================] - 3s 81us/step - loss: 0.1777 - acc: 0.9504 - val_loss: 0.1907 - val_acc: 0.9463
Epoch 7/50
31500/31500 [==============================] - 3s 81us/step - loss: 0.1577 - acc: 0.9571 - val_loss: 0.1734 - val_acc

In [17]:
#predict
y_hat= model.predict(X_test)

In [21]:
df =pd.DataFrame(y_hat)
df.index = df.index + 1

In [22]:
df['max'] = df.idxmax(axis=1)

In [23]:
#ready for kaggle
df[['max']].index.name = 'ImageId'
df[['max']].rename({'max': 'Label'},axis=1)

,Label
ImageId,
1,9
2,2
3,1
4,1
5,9
6,4
7,8
8,7
9,7


In [24]:
test = pd.read_csv('./test.csv')

In [25]:
test/=255

In [26]:
test.shape

(28000, 784)

In [27]:
y_hat = model.predict(test)

In [32]:
tested =pd.DataFrame(y_hat)
tested.index = tested.index +1

In [33]:
tested['max'] = tested.idxmax(axis=1)

In [34]:
tested['max']

1        2
2        0
3        9
4        9
5        3
6        7
7        0
8        3
9        0
10       3
11       5
12       7
13       4
14       0
15       4
16       3
17       3
18       1
19       9
20       0
21       9
22       1
23       1
24       5
25       7
26       4
27       2
28       7
29       4
30       7
        ..
27971    5
27972    0
27973    4
27974    8
27975    0
27976    3
27977    6
27978    0
27979    1
27980    9
27981    3
27982    1
27983    1
27984    0
27985    4
27986    5
27987    2
27988    2
27989    9
27990    6
27991    7
27992    6
27993    1
27994    9
27995    7
27996    9
27997    7
27998    3
27999    9
28000    2
Name: max, Length: 28000, dtype: int64

In [45]:
#submit to kaggle
tested[['max']].index.name = 'ImageId'
tested['ImageId'] = tested.index
tested
submit = tested[['ImageId','max']].rename({'max': 'Label'},axis=1)
submit

,ImageId,Label
ImageId,,
1,1,2
2,2,0
3,3,9
4,4,9
5,5,3
6,6,7
7,7,0
8,8,3
9,9,0


In [46]:
submit.to_csv('./submit.csv',index=False)

# CNN
Convolutional Neural Net

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=124)

In [86]:
X_train = X_train.values.reshape(X_train.shape[0], 28,28,1)
X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)


In [87]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [88]:
X_train /= 255
X_test /= 255

In [89]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [213]:
model = Sequential()

In [214]:
#convolution layers
model.add(Convolution2D(filters =16,
                       kernel_size = 3,
                       activation ='relu',
                       input_shape = (28,28,1)))
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#convolution layer
model.add(Convolution2D(filters =20,
                       kernel_size = 3,
                       activation ='relu'
                       ))
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#flatten layer
model.add(Flatten())
#dense layers
model.add(Dense(150, activation='relu'))
#output layer
model.add(Dense(10, activation='softmax'))

In [215]:
#compile
model.compile(loss ='categorical_crossentropy', optimizer='adam', metrics=['acc'] )
early_stop = EarlyStopping(monitor='val_loss', min_delta=0)

In [216]:
#fit
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs =50, batch_size=2000, callbacks=[early_stop])

Train on 31500 samples, validate on 10500 samples
Epoch 1/50
31500/31500 [==============================] - 11s 338us/step - loss: 1.9931 - acc: 0.4954 - val_loss: 1.4562 - val_acc: 0.7004
Epoch 2/50
31500/31500 [==============================] - 10s 315us/step - loss: 0.9485 - acc: 0.7736 - val_loss: 0.5732 - val_acc: 0.8293
Epoch 3/50
31500/31500 [==============================] - 10s 315us/step - loss: 0.4474 - acc: 0.8661 - val_loss: 0.3702 - val_acc: 0.8866
Epoch 4/50
31500/31500 [==============================] - 10s 315us/step - loss: 0.3208 - acc: 0.9034 - val_loss: 0.2830 - val_acc: 0.9159
Epoch 5/50
31500/31500 [==============================] - 10s 322us/step - loss: 0.2548 - acc: 0.9236 - val_loss: 0.2337 - val_acc: 0.9301
Epoch 6/50
31500/31500 [==============================] - 10s 319us/step - loss: 0.2136 - acc: 0.9372 - val_loss: 0.1992 - val_acc: 0.9392
Epoch 7/50
31500/31500 [==============================] - 10s 320us/step - loss: 0.1804 - acc: 0.9471 - val_loss: 0.

In [217]:
test = pd.read_csv('./test.csv')

In [218]:
test = test.values.reshape(test.shape[0], 28, 28, 1)

In [219]:
test = test.astype('float32')

In [220]:
test /= 255

In [221]:
y_hat = model.predict_classes(test)

In [222]:
y_hat

array([2, 0, 9, ..., 3, 9, 2])

In [223]:
tested =pd.DataFrame(y_hat)
tested.index = tested.index +1

In [224]:
tested.rename({0:'Label'}, axis =1, inplace=True)
tested['ImageId'] = tested.index


In [225]:
submit2 = tested[['ImageId','Label']]

submit2

,ImageId,Label
1,1,2
2,2,0
3,3,9
4,4,9
5,5,3
6,6,7
7,7,0
8,8,3
9,9,0
10,10,3


In [226]:
#submit to kaggle
submit2.to_csv('./submit2.csv', index=False)